![Digital Earth Pacific](dep.png)

In [ ]:
import geopandas as gpd
import joblib
import numpy as np
import odc.geo.xr  # noqa
from dask import config
from dask.distributed import Client as dask_client
from odc.stac import load
from pystac_client import Client
from shapely import geometry
import depal_fj as dep
from depal_fj import predict_xr
from model import get_overlay
from ipyleaflet import Map, LayersControl, basemaps

In [ ]:
%reload_ext autoreload
%autoreload 2

config.set({'distributed.worker.daemon': False})

## Find and load data

Load data and set up your array to use for prediction

In [ ]:
# Configure some things up front
chunks = dict(x=2048, y=2048)
year = "2023"
model_file_name = "fj_lulc.model"

aoi = dep.get_island("Viti Levu")
bbox = dep.get_bbox(aoi)
bbox_geometry = geometry.box(*bbox)

gdf = gpd.GeoDataFrame({'geometry': [bbox_geometry]}, crs='EPSG:4326')
gdf.explore()

In [ ]:
catalog = "https://stac.staging.digitalearthpacific.org"
client = Client.open(catalog)

# Search for Sentinel-2 GeoMAD data
items = client.search(
    collections=["dep_s2_geomad"],
    bbox=bbox,
    datetime=year
).items()

# Load the data
data = load(items, chunks=chunks, bbox=bbox, resolution=10).squeeze("time")

#coastal clip
data = dep.do_coastal_clip(aoi, data, buffer=0)
data

In [ ]:
loaded_model = joblib.load(model_file_name)

## Run the prediction

In [ ]:
filled = data.fillna(-9999.0)

# This runs the actual prediction
with dask_client():
    predicted = predict_xr(loaded_model, filled, proba=True)

    # Convert to int
    cleaned_predictions = predicted.copy(deep=True)
    cleaned_predictions.predictions.data = predicted.predictions.data.astype(np.int8)
    cleaned_predictions.probabilities.data = predicted.probabilities.data.astype(np.float32)

    cleaned_predictions = cleaned_predictions.rename(
        {"predictions": "lulc", "probabilities": "prob"}
    ).compute()

cleaned_predictions

In [ ]:
from matplotlib import colors

classes = [
    [1, "Cropland", "#ffce33"],
    [2, "Forest", "#064a00"],
    [3, "Grassland", "#d7ffa0"],
    [4, "Settlement", "#d10a1e"],
    [5, "Mangroves", "#07b28d"],
    [6, "Water", "#71a8ff90"],
    [7, "Bare_Other", "#968640"]
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [8]
norm = colors.BoundaryNorm(bounds, c_map.N)

cleaned_predictions.lulc.plot.imshow(cmap=c_map, norm=norm, size=10)

### Interactive Display

In [ ]:
# Write GeoTIFF
cleaned_predictions.lulc.odc.write_cog("fj_lulc_" + str(year) + ".tif", overwrite=True)